In [5]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

In [6]:
X_509_w_outliers = np.load('509_data/R_509_1_data.npy')
X_509_obi_long = np.load('509_data/R_509_1_data_obi.npy')
y_509_w_outliers = np.load('509_data/R_509_1_labels.npy')
y_509_obi_long = np.load('509_data/R_509_1_labels_new.npy')

In [7]:
def remove_outliers(X, y, outlier_quantile=.99):

    # remove windows that in the top 1% of RMS values 
    rms_thresh = np.quantile(y[:,2], outlier_quantile) 
    rms_outlier_ind = np.argwhere(y[:,2]> rms_thresh)
    return np.delete(X, rms_outlier_ind, axis=0), np.delete(y, rms_outlier_ind, axis=0)

X_509, y_509 = remove_outliers(X_509_w_outliers, y_509_w_outliers)

In [8]:
y_509_obi = y_509_obi_long[2:-2] 
X_509_obi = X_509_obi_long[2:-2]
assert y_509_obi.shape[0] == y_509.shape[0], print("Shape mismatch")


In [9]:
def mark_candidates_and_TNES_A(y):

    # convert features into percentiles 
    rel_quantile = stats.rankdata(y[:, 0],
                                        "average")/y.shape[0]
    corr_quantile = stats.rankdata(y[:, 1],
                                            "average")/y.shape[0]
    rms_quantile = stats.rankdata(y[:, 2],
                                        "average")/y.shape[0]
    quantiles =  np.vstack((rel_quantile, corr_quantile, rms_quantile)).T

    scores = np.average(quantiles, axis=1)

    candidates = np.argwhere(scores > .65)
    
    candidate_quantiles = quantiles[candidates].squeeze()

    A_scores = np.average(candidate_quantiles, weights=[1/5., 3/5., 1/5.], axis=1)

    TNEs = np.argwhere(A_scores > .9)

    return candidates, TNEs

cand_509_inds, TNE_A_509_inds = mark_candidates_and_TNES_A(y_509)
cand_509_inds_obi, TNE_A_509_inds_obi = mark_candidates_and_TNES_A(y_509_obi)

In [80]:
X_cand_509_obi = X_509_obi[cand_509_inds_obi].squeeze()
y_cand_509_obi = np.zeros(cand_509_inds_obi.shape[0])
y_cand_509_obi[TNE_A_509_inds_obi] = 1

fourth = int(X_cand_509_obi.shape[0] / 4)

X_cand_509_obi_first_fourth = X_cand_509_obi[:fourth]
y_cand_509_obi_first_fourth = y_cand_509_obi[:fourth]

np.save('509_data/X_q1_A_obi.npy', X_cand_509_obi_first_fourth)
np.save('509_data/y_q1_A_obi.npy', y_cand_509_obi_first_fourth)

In [39]:
print(TNE_A_509_inds.shape, TNE_A_509_inds_new.shape)

(14891, 1) (14442, 1)


In [2]:
R_train = np.load('prepared_data/R_dataset_X_train.npy')
R_y_A = np.load('prepared_data/R_dataset_y_train_A.npy')